In [ ]:
import random

import boto3

session = boto3.Session(profile_name="boto3_101")

Using the aws `client`


In [5]:
rn = random.randrange(10**5, 10**6)

s3_client = session.client("s3")
s3_client.create_bucket(Bucket=f"boto3-bucket-{rn}")

{'ResponseMetadata': {'RequestId': 'CVAQDQ1A12H0KWAF',
  'HostId': 'xKsnkwXJ/uM0wgG+U/kAlo1nyj9D+UYuGP6aw8I/0Ok2n20gILIdPLjhPudAUIOqyEcUiwsiirBsG1JBVkyy2Xi28Jg7py0l',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xKsnkwXJ/uM0wgG+U/kAlo1nyj9D+UYuGP6aw8I/0Ok2n20gILIdPLjhPudAUIOqyEcUiwsiirBsG1JBVkyy2Xi28Jg7py0l',
   'x-amz-request-id': 'CVAQDQ1A12H0KWAF',
   'date': 'Mon, 17 Mar 2025 09:10:44 GMT',
   'location': '/boto3-bucket-298564',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/boto3-bucket-298564'}

In [6]:
s3_client.delete_bucket(Bucket="boto3-bucket-298564")

{'ResponseMetadata': {'RequestId': 'MJ2WJP7ZBX0B185Q',
  'HostId': 'MVApgEZ5cwWbDcTY42LZ1Pbpg0zZhCMM7tBGOJ/HXowFofELB3Y4AE1s+XjnUvCmArCQ7JBJWhxKbZ1F0X/Tde2J9rNkbUlaacAOk+ygt7M=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'MVApgEZ5cwWbDcTY42LZ1Pbpg0zZhCMM7tBGOJ/HXowFofELB3Y4AE1s+XjnUvCmArCQ7JBJWhxKbZ1F0X/Tde2J9rNkbUlaacAOk+ygt7M=',
   'x-amz-request-id': 'MJ2WJP7ZBX0B185Q',
   'date': 'Mon, 17 Mar 2025 09:10:53 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [7]:
s3_client.list_buckets()["Buckets"]

[{'Name': 'add-demo-cloudfront-v1',
  'CreationDate': datetime.datetime(2024, 6, 18, 12, 47, 1, tzinfo=tzutc())},
 {'Name': 'add-demo-s3-v1',
  'CreationDate': datetime.datetime(2024, 6, 14, 16, 26, 57, tzinfo=tzutc())},
 {'Name': 'sagemaker-studio-891377239273-vees33vfkeb',
  'CreationDate': datetime.datetime(2024, 10, 25, 5, 35, 20, tzinfo=tzutc())},
 {'Name': 'sagemaker-us-east-1-891377239273',
  'CreationDate': datetime.datetime(2024, 10, 25, 5, 35, 25, tzinfo=tzutc())},
 {'Name': 'terraform-backend-frz9g',
  'CreationDate': datetime.datetime(2024, 9, 11, 9, 35, 37, tzinfo=tzutc())}]

In [8]:
list_s3_buckets = lambda: [
    bucket["Name"] for bucket in s3_client.list_buckets()["Buckets"]
]
list_s3_buckets()

['add-demo-cloudfront-v1',
 'add-demo-s3-v1',
 'sagemaker-studio-891377239273-vees33vfkeb',
 'sagemaker-us-east-1-891377239273',
 'terraform-backend-frz9g']

In [ ]:
def bucket_name_generator(prefix: str = "boto3-bucket", random_digits: int = 6) -> str:
    """Generate random names for aws s3 buckets in format: "prefix-random_digits",
    then create state that holds bucket names generated via `.names` attr.

    Parameters
    ----------
    `prefix` : str
        Bucket name prefix.
    `random_digits` : int
        Bucket name suffix i.e. generated random number with specified number of digits.

    Returns
    -------
    str
        Bucket name.
    """
    rn = random.randrange(10 ** (random_digits - 1), 10 ** (random_digits))
    bucket_name = f"{prefix}-{rn}"

    # create state that holds bucket names generated
    if not hasattr(bucket_name_generator, "names"):
        bucket_name_generator.names = []
    bucket_name_generator.names.append(bucket_name)

    return bucket_name

In [ ]:
def create_buckets(
    number_of_buckets: int, bucket_prefix: str, random_digits: int
) -> None:
    """Create aws s3 buckets with in format "prefix-random_digits" in `num` number of times.

    Parameters
    ----------
    `number_of_buckets` : int
        Number of buckets to create in AWS S3 through iteration.
    `bucket_prefix` : str
        Bucket name prefix.
    `random_digits` : int
        Bucket name suffix i.e. generated random number with specified number of digits.

    Returns
    -------
    None
    """
    for _ in range(number_of_buckets):
        s3_client.create_bucket(
            Bucket=bucket_name_generator(
                prefix=bucket_prefix, random_digits=random_digits
            )
        )

In [14]:
create_buckets(3, "boto3-bucket", 4)

In [15]:
bucket_name_generator.names

['boto3-bucket-8897', 'boto3-bucket-7732', 'boto3-bucket-4849']

In [ ]:
def delete_buckets(bucket_names: list, contains_name: str | None = None) -> list:
    """Delete aws s3 buckets from generated bucket names and/or buckets containing the name in `contains_name` param.

    Also cleans the `bucket_names` container after bucket deletion.

    Parameters
    ----------
    `bucket_names` : list
        Buckets names to be deleted from AWS S3 bucket. These bucket names must already exist in AWS S3.
    `contains_name` : str | None
        Delete bucket names not included in `bucket_names`. Defaults to `None`

    Returns
    -------
    list
        Current AWS S3 buckets.
    """

    # delete buckets from `bucket_names` seq
    for _, bucket_name in enumerate(bucket_names):
        s3_client.delete_bucket(Bucket=bucket_name)

    s3_buckets = lambda: [
        bucket["Name"] for bucket in s3_client.list_buckets()["Buckets"]
    ]

    # delete buckets using `contains_name` value
    if contains_name:
        for bucket_name in s3_buckets():
            if contains_name in bucket_name:
                s3_client.delete_bucket(Bucket=bucket_name)
            else:
                raise ValueError(f"{contains_name} is not in S3 buckets.")

    bucket_names.clear()

    return s3_buckets()

In [17]:
list_s3_buckets()

['add-demo-cloudfront-v1',
 'add-demo-s3-v1',
 'boto3-bucket-4849',
 'boto3-bucket-7732',
 'boto3-bucket-8897',
 'sagemaker-studio-891377239273-vees33vfkeb',
 'sagemaker-us-east-1-891377239273',
 'terraform-backend-frz9g']

In [18]:
bucket_name_generator.names

['boto3-bucket-8897', 'boto3-bucket-7732', 'boto3-bucket-4849']

In [19]:
delete_buckets(bucket_name_generator.names)

['add-demo-cloudfront-v1',
 'add-demo-s3-v1',
 'sagemaker-studio-891377239273-vees33vfkeb',
 'sagemaker-us-east-1-891377239273',
 'terraform-backend-frz9g']

In [20]:
bucket_name_generator.names

[]

Using the aws `resource`
